# Sensitivity Analysis for the TTI-explorer

## Preliminaries

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tti import simulation
from emukit.test_functions.sensitivity import Ishigami
from emukit.core import ContinuousParameter, DiscreteParameter, ParameterSpace

np.random.seed(10) # for reproducibility

PROB_DOMAIN = (0,1)

## Initial Sample

First, we define the parameter space over which we want to perform the sensitivity analysis and take an initial sample from this using the Latin hypercube sampling method.

In [5]:
space = ParameterSpace(
            [
                ContinuousParameter('go_to_school_prob', *PROB_DOMAIN),
                DiscreteParameter('testing_delay', [*range(6)]),
                DiscreteParameter('quarantine_length', [*range(1,11)])
            ])

config_details = {
                    0:  {"name": "go_to_school_prob", "config": "policy_config"},
                    1:  {"name": "testing_delay", "config": "policy_config"},
                    2:  {"name": "quarantine_length", "config": "policy_config"}
                 }

In [ ]:
from emukit.core.initial_designs import LatinDesign

def initial_sample():
    num_initial_points = 50
    design = LatinDesign(space)
    x = design.get_samples(num_initial_points)
    y = np.array([total_cost_R(k) for k in x])[:,np.newaxis]
    
    return x, y

## Emulator model

Then, set up the emulator - here using GP regression - to approximate the output of the simulation. Initialise using the initial samples obtained above.

In [ ]:
def setup_model(x, y):
    # Use GP regression as the emulator
    from GPy.models import GPRegression
    from GPy.kern import RBF, Bias
    from emukit.model_wrappers import GPyModelWrapper

    kern = RBF(len(space.parameters), variance=1.0, lengthscale=100, ARD=True) + Bias(len(space.parameters))
    model_gpy = GPRegression(x, y, kern)
    model_emukit = GPyModelWrapper(model_gpy)
    model_emukit.optimize()
    
    return model_emukit, model_gpy

## Experimental Design Loop

Set up an ED loop using uncertainty sampling (US) - which corresponds to using the ModelVariance acqusition function. Run this loop for a numbre of iterations.

In [8]:
from emukit.experimental_design.acquisitions import ModelVariance
from emukit.experimental_design import ExperimentalDesignLoop

def sim_loop(x):
    return np.array([[simulation(x[0], config_details)['Effective R']]])

def run_exp_loop(model_emukit):
    model_variance = ModelVariance(model = model_emukit)        
    exp_loop = ExperimentalDesignLoop(model = model_emukit,
                                    space = space,
                                    acquisition = model_variance,
                                    update_interval = 1,
                                    batch_size = 1)
    exp_iterations = 100
    exp_loop.run_loop(sim_loop, exp_iterations)
    return

## Sensitivity Analysis

Perform the sensitivity analysis. We use SALib here because emukit does not fully support SA over discrete parameters.

In [10]:
from SALib.sample import saltelli
from SALib.analyze import sobol

def sens_analysis(model_emukit):
    # Define the parameter space
    problem = {
      'num_vars': 3,
      'names': ['go_to_school_prob', 'testing_delay', 'quarantine_length'],
      'bounds': [[0, 1], [0, 6], [1, 11]]
    }

    # Generate samples
    param_values = saltelli.sample(problem, 10000)
    # Round discrete parameters to integers
    param_values[:,1] = np.around(param_values[:,1])
    param_values[:,2] = np.around(param_values[:,2])

    Y = model_emukit.predict(param_values)[0][:,0]

    # Perform analysis
    Si = sobol.analyze(problem, Y, print_to_console=True)

    main_effects_sa = Si['S1']
    total_effects_sa = Si['ST']
    return main_effects_sa, total_effects_sa

In [11]:
def run_sensitivity():
    x_init, y_init = initial_sample()
    model_emukit, model_gpy = setup_model(x_init, y_init)
    run_exp_loop(model_emukit)
    
    main_effects_sa, total_effects_sa = sens_analysis(model_emukit)
    
    return main_effects_sa, total_effects_sa

In [12]:
def plot_sens(name, main_effects_sa, total_effects_sa):
    param_names = [x.name for x in space.parameters]
    bar_width = 0.35
    label_locs = np.arange(len(param_names))

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(label_locs - bar_width/2, main_effects_sa, bar_width, label='First-order Sobol indices')
    ax.bar(label_locs + bar_width/2, total_effects_sa, bar_width, label='Total effects')

    ax.set_ylabel('% of explained output variance')
    ax.set_xticks(label_locs)
    ax.set_xticklabels(param_names)
    ax.legend()

    fig.savefig('plots/' + name + '.eps', format='eps')

In [ ]:
me, te = run_sensitivity()
plot_sens('R_3', me, te)